# File format conversion
CMG sr3/gmch.sr3 --> CMG rwo (by running CMG Results Report software on rwd files) --> numpy array  
sr3 is the CMG simulation result file, whereas rwo is the extracted info in the ascii format

# Step 1: Convert CMG sr3/gmch.sr3 files to rwo files

In [ ]:
from pathlib import Path
from CMG2npy import generate_CMG_rwd, run_CMG_rwd_report
from tqdm import tqdm

# user inputs
property = 'STRESMXP'; name_prefix = '250915'; n_cases = 90

# setup paths
base_path = Path('.')
sim_folder_path = base_path/f'{name_prefix}_sherlock'
for case_num in tqdm(range(1,n_cases+1),desc='Converting sr3 to rwo'):
    generate_CMG_rwd(
        sr3_folder_path = sim_folder_path,
        case_name = f'case{case_num}',
        property = property,
        is_gmc_property = True,
        precision = 4
    )

    run_CMG_rwd_report(
        rwd_folder_path = sim_folder_path,
        case_name = f'case{case_num}',
        cmg_version = 'ese-ts2win-v2024.20',
    )

    # remove rwd files
    rwd_folder_path = sim_folder_path
    Path(rwd_folder_path, f'case{case_num}.rwd').unlink()

print("\nFinished generating rwo files for all cases.")

# Step 2: Extract simulation results from rwo files into numpy arrays

## Option 1: Extract results on all grid cells

In [ ]:
from pathlib import Path
from pathlib import Path
from CMG2npy import CMG_rwo2npy
from tqdm import tqdm

# user inputs
property = 'STRESMXP'; name_prefix = '250915'; n_cases = 90

# set up paths
base_path = Path('.')
sim_folder_path = base_path/f'{name_prefix}_sherlock'
save_folder_path = sim_folder_path/f'{name_prefix}_gmc'
save_folder_path.mkdir(parents=True, exist_ok=True)

for case_num in tqdm(range(1,n_cases+1), desc='Generating numpy arrays from CMG cases'):
    sim_results = CMG_rwo2npy(
        rwo_folder_path = sim_folder_path/'rwo',
        case_name = f'case{case_num}',
        property = property,
        is_save = True,
        save_folder_path = save_folder_path,
        show_info = False
    )

print("\nFinished generating numpy arrays for all cases.")

## Option 2: Only keep fault grid cells and simulated layers

1. For the JD_Sula_2025_flow grid (107x117x79), the reservoir layers are k=41-79. The other layers were not simulated by setting ntg=0.
2. For the JD_Sula_2025_gmc grid (107x117x10), the reservoir layers are k=6-10. 

In [ ]:
import numpy as np
from pathlib import Path
from CMG2npy import CMG_rwo2npy
from tqdm import tqdm

# user inputs
property = 'STRESMXP'; name_prefix = '250915'; n_cases = 90
coor_fault = np.load('JD_Sula_2025_gmc_coor&fault.npy') # load the grid containing fault id

# set up paths
base_path = Path('.')
sim_folder_path = base_path/f'{name_prefix}_sherlock'
save_folder_path = sim_folder_path/f'{name_prefix}_gmc'
save_folder_path.mkdir(parents=True, exist_ok=True)

for case_num in tqdm(range(1,n_cases+1),desc='Generating numpy arrays from CMG cases'):
    # extract results on all grid cells
    sim_results = CMG_rwo2npy(
        rwo_folder_path = sim_folder_path/'rwo',
        case_name = f'case{case_num}',
        property = property,
        is_save = False,
        save_folder_path = save_folder_path,
        show_info = False
    )

    # keep results on fault grid cells only
    nan_mask = np.isnan(coor_fault[:,:,:,3])
    sim_results[nan_mask] = np.nan

    # keep results of simulated layers only
    # Python indices are 0-based, to extract slices 41–79, use 40:79
    sim_results_trimmed = sim_results[:, :, 5:10]

    # save
    np.save(save_folder_path/f'case{case_num}_{property}.npy',sim_results_trimmed)


print("\nFinished generating numpy arrays for all cases.")